In [1]:
import pandas as pd
import tkinter as tk
from PIL import ImageTk, Image
import requests
from io import BytesIO
import random

#Function to search for movies and update the display
def search_movies():
    query = search_entry.get().strip()  #Gets the search query and remove leading/trailing spaces
    if query:
        #Perform the search and display the matching movies
        mask = df['title'].str.contains(query, case=False, na=False)
        results = df[mask]
        display_movies(results)
    else:
        #Clear the display area if the search query is empty
        display_text.delete(1.0, tk.END)

#Function to display the top 10 highest-rated movies
def display_top_10_imdb():
    top_10 = df.nlargest(10, 'imdb_score')
    top_10['Rank'] = range(1, 11)
    display_movies(top_10)
    
#Function to display the top 10 highest-rated movies
def display_top_10_tmdb():
    top_10 = df.nlargest(10, 'tmdb_score')
    top_10['Rank'] = range(1, 11)
    display_movies(top_10)    

#Function to display movies by genre, ordered by highest 'imdb_score'
def display_movies_by_genre(genre):
    genre_movies = df[df['genres'].str.contains(genre, case=False, na=False)]
    genre_movies = genre_movies.sort_values(by='imdb_score', ascending=False)
    display_movies(genre_movies)

#Function to display the movie description page
def display_description(title):
    movie = df[df['title'] == title]
    description = movie['description'].values[0]

    #Create the description page
    description_window = tk.Toplevel(window)
    description_window.title("Movie Description")

    description_label = tk.Label(description_window, text="Description:")
    description_label.pack()

    description_text = tk.Text(description_window)
    description_text.insert(tk.END, description)
    description_text.pack()

    #Create the back button
    back_button = tk.Button(description_window, text="Back", command=description_window.destroy)
    back_button.pack()

#Function to display the movie results
def display_movies(results):
    display_text.delete(1.0, tk.END)
    for index, row in results.iterrows():
        title = row['title']
        imdb_score = row['imdb_score']
        tmdb_score = row['tmdb_score']
        
        #Format 'imdb_score' and 'tmdb_score' to one decimal place
        imdb_score = '{:.1f}'.format(imdb_score)
        tmdb_score = '{:.1f}'.format(tmdb_score)

        #Format of Title and Ratings
        movie_text = f"{title}\nIMDb Score: {imdb_score}\nTMDb Score: {tmdb_score}\n"
        display_text.insert(tk.END, movie_text)

        #Create the movie result with view description button
        view_button = tk.Button(display_text, text="View Description", command=lambda title=title: display_description(title))
        display_text.window_create(tk.END, window=view_button)
        display_text.insert(tk.END, "\n")

        #Apply bold formatting to the movie title
        start_index = str(display_text.search(title, "1.0", stopindex=tk.END))
        end_index = f"{start_index}+{len(title)}c"
        display_text.tag_add("bold", start_index, end_index)
        display_text.tag_config("bold", font=("bold",))
        
        #Apply bold formatting to the IMDb score
        start_index = str(display_text.search(str(imdb_score), end_index, stopindex=tk.END))
        end_index = f"{start_index}+{len(str(imdb_score))}c"
        display_text.tag_add("bold", start_index, end_index)
        display_text.tag_config("bold", font=("bold",))
        
        #Apply bold formatting to the TMDb score
        start_index = str(display_text.search(str(tmdb_score), end_index, stopindex=tk.END))
        end_index = f"{start_index}+{len(str(tmdb_score))}c"
        display_text.tag_add("bold", start_index, end_index)
        display_text.tag_config("bold", font=("bold",))

#URL of the CSV file on GitHub
csv_url = 'https://raw.githubusercontent.com/CenturionRose/Flixrate/JustWatch-Data/titles2.csv'

#Reads the CSV file from the online repository
df = pd.read_csv(csv_url)

#Creates the main window
window = tk.Tk()
window.title("Movie Search")

#Adds the title image
#URL of the image file on GitHub
image_url = 'https://raw.githubusercontent.com/CenturionRose/Flixrate/f7873f96f75958552659f8059ed9c9495bb710ac/FlixRateLogo.png'
#Request the image file from the URL
response = requests.get(image_url)
image_data = response.content

#Converts the image data to a PIL Image object
image = Image.open(BytesIO(image_data))

#Creates a Tkinter PhotoImage from the PIL Image
photo = ImageTk.PhotoImage(image)

#Creates a label to display the image
image_label = tk.Label(window, image=photo)
image_label.pack()

#Creates the search bar
search_label = tk.Label(window, text="Search title:")
search_label.pack()

search_entry = tk.Entry(window)
search_entry.pack()

search_button = tk.Button(window, text="Search", command=search_movies)
search_button.pack()

#Creates the filter button for top 10 movies
filter_button = tk.Button(window, text="Top 10 Highest Rated IMDb", command=display_top_10_imdb)
filter_button.pack(side=tk.TOP, anchor=tk.NE)

#Creates the filter button for top 10 movies
filter_button2 = tk.Button(window, text="Top 10 Highest Rated TMDb", command=display_top_10_tmdb)
filter_button2.pack(side=tk.TOP, anchor=tk.NE)

#Function to show or hide the genre buttons
def show_genre_buttons():
    global genre_buttons_visible

    if genre_buttons_visible:
        for genre in genres:
            genre_buttons[genre].pack_forget() #Hides the genre button
        genre_buttons_visible = False
    else:
        genre_row = 5  #Starting row for genre buttons
        genre_col = 0  #Starting column for genre buttons
        for genre in genres:
            genre_buttons[genre].pack(side=tk.LEFT)
            genre_col += 1 #Increment the column counter
            if genre_col > 3: #If the column counter exceeds 3
                genre_row += 1 #Move to the next row
                genre_col = 0 #Reset the column counter
        genre_buttons_visible = True 
        
#Create the genre button
genre_button = tk.Button(window, text="Genres", command=show_genre_buttons)
genre_button.pack(side=tk.TOP, anchor=tk.NE)

#Creates a dictionary of genres and their buttons
genres = ['documentation', 'action', 'animation', 'comedy', 'drama', 'crime', 'fantasy', 'horror',
          'scifi', 'western', 'romance', 'european', 'history', 'thriller', 'sport', 'war', 'music', 'reality']
genre_buttons = {}
genre_buttons_visible = False

for genre in genres:
    genre_buttons[genre] = tk.Button(window, text=genre, command=lambda genre=genre: display_movies_by_genre(genre))
    
#Function to clear the displayed content
def clear_content():
    display_text.delete(1.0, tk.END)
    
#Creates the clear button
clear_button = tk.Button(window, text="Clear", command=clear_content)
#Pack the clear button in the top right corner
clear_button.pack(side=tk.TOP, anchor=tk.NW)

#Function to display a random movie
def display_random_movie():
    random_row = df.sample(n=1)
    display_movies(random_row)

#Creates the random button
random_button = tk.Button(window, text="Random Movie", command=display_random_movie)
random_button.pack(side=tk.TOP, anchor=tk.NE)

#Creates the display area
display_text = tk.Text(window)
display_text.pack(expand=True, fill=tk.BOTH)

#Starts the application
window.mainloop()